In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlretrieve, quote
from urllib.parse import urljoin
import urllib.request
from openpyxl import load_workbook
from tqdm import tqdm_notebook

# Источник 1:
- Основные показатели в сферах телекоммуникации
- Использование цифровых технологий в РФ 

In [3]:
url = 'https://rosstat.gov.ru/statistics/science'
connect_xls_url = 'https://rosstat.gov.ru'
data = requests.get(url).text

### Получение ссылок на актуальные файлы

In [4]:
soup = BeautifulSoup(data, 'html.parser')

In [5]:
def get_links_dict(soup):
  xls_blocks = soup.find_all('div',{'class':'document-list__item document-list__item--row'})
  dimSource = {}

  for idx in range(len(xls_blocks)):
    title = xls_blocks[idx].find('div',{'class':'document-list__item-title'}).get_text().replace("\n", "").strip()
    content = xls_blocks[idx].find('a',{'class':'btn btn-icon btn-white btn-br btn-sm'}).get('href')
    download_link = connect_xls_url + content
    dimSource[title] = download_link

  return dimSource

In [6]:
dimSource = get_links_dict(soup)

### Загрузка данных и обработка в pandas

In [7]:
#telecom - Основные показатели в сферах телекоммуникации
#tech - Использование цифровых технологий в РФ

df_telecom_name = 'Основные показатели в сфере телекоммуникаций (по данным Минцифры России и Росстата)'
df_tech_name = 'Использование цифровых технологий организациями по Российской Федерации, субъектам Российской Федерации и видам экономической деятельности (с 2003 г.)'

In [8]:
years = list(range(1990,2022))

def parse_df(link, sheet_name, skiprows, skipfooter, df_name):
  df = pd.read_excel(
    link, 
    sheet_name=sheet_name, 
    skiprows=skiprows,
    skipfooter=skipfooter
  )

  df.replace({'...': None}, inplace=True)

  df.columns = ['region'] + list(df.columns[1:-1]) + ['place']
  df.region = df.region.str.replace('\n', ' ')
  df['value_type'] = df_name
  
  for year in years:
    if year not in df.columns:
      df[year] = None

  df = df[1:].reset_index(drop=True)
  df = df[['region', 'value_type', 'place'] + years]

  df.dropna(axis=0, subset=df.columns[3:], how='all', inplace=True)

  return df

In [9]:
df_telecom_1 = parse_df(
    dimSource[df_telecom_name], 
    sheet_name=1, 
    skiprows=5, 
    skipfooter=5, 
    df_name='telecom_usage_per_person'
)

df_telecom_2 = parse_df(
    dimSource[df_telecom_name], 
    sheet_name=6, 
    skiprows=5, 
    skipfooter=5, 
    df_name='telecom_phone_count_per_1000'
)

df_tech_usage = parse_df(
    dimSource[df_tech_name], 
    sheet_name=1, 
    skiprows=5, 
    skipfooter=5, 
    df_name='tech_usage'
)

In [10]:
telecom_tech = pd.concat([df_telecom_1, df_telecom_2, df_tech_usage], axis=0).sort_values('region')

In [12]:
telecom_tech.sample(10)

,region,value_type,place,1990,1991,1992,1993,1994,1995,1996,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
79,Кемеровская область,telecom_usage_per_person,64.0,0.013,0.018,0.137,2.037,10.282,31.126,65.321,...,4213.377,4455.493,4382.932,4194.188,4403.9,4445.6,4421.8,4655.000,4736.8,4986.8
47,Республика Марий Эл,telecom_phone_count_per_1000,1823.1,None,None,None,None,None,None,None,...,1611.9,1737.9,1861.9,1838.9,1846.6,1818.3,1758.1,1871.366,1785.6,None
65,Ямало-Ненецкий автономный округ,telecom_phone_count_per_1000,2435.7,None,None,None,None,None,None,None,...,2464.6,2570.7,2469.0,2683.7,2609.5,2314.2,2303.4,2441.961,2425.2,None
40,Кабардино-Балкарская Республика,telecom_phone_count_per_1000,1329.4,None,None,None,None,None,None,None,...,1305.8,1444.5,1405.2,1388.4,1297.8,1380.2,1316.8,1405.793,1343.1,None
29,Республика Адыгея,telecom_phone_count_per_1000,993.3,None,None,None,None,None,None,None,...,1226.5,1343.6,1295.9,1154.5,1114.1,1050.2,1030.4,1060.718,992.6,None
48,Ставропольский край,telecom_usage_per_person,36.0,0.021,0.025,0.133,1.872,10.9,37.986,62.561,...,4443.402,4637.305,4636.843,4565.004,4525.4,4826.8,5205.6,5511.300,5602.6,5814.0
10,Московская область,telecom_usage_per_person,NaN,…,…,…,…,…,…,…,...,…,…,…,…,NaN,NaN,NaN,NaN,NaN,NaN
67,Тюменская область,telecom_usage_per_person,NaN,0.021,0.03,0.236,3.115,26.483,88.595,192.707,...,8810.722,9167.135,8830.336,8575.131,7757.6,7639.9,7086.7,6937.100,6924.7,7048.6
26,Псковская область,telecom_phone_count_per_1000,1946.0,None,None,None,None,None,None,None,...,1731.5,1749.7,1725.3,1820,1819.5,1910.3,1861.5,1868.336,1881.2,None
19,Республика Карелия,telecom_phone_count_per_1000,2014.6,None,None,None,None,None,None,None,...,1874.7,1860.6,1753.4,1818.4,1769.0,1858.6,1794.8,1866.348,1888.1,None


In [13]:
final_df = telecom_tech[['region', 'value_type', 'place', years[0]]].rename(columns={years[0]: "value"})
final_df['year'] = years[0]
for x in years[1:]:
  current_df = telecom_tech[['region', 'value_type', 'place', x]].rename(columns={x: "value"})
  current_df['year'] = x
  final_df = pd.concat([final_df, current_df])

In [14]:
telecom_to_load = final_df[~final_df['region'].str.contains('округ')]

# Источник 2:


*   Средняя заработная плата по регионам




In [15]:
url = 'https://rosstat.gov.ru/labor_market_employment_salaries'
data = requests.get(url).text

In [16]:
soup = BeautifulSoup(data, 'html.parser')

In [17]:
# по данному датасету отличается структура тэгов
def get_links_dict_2(soup):
  xls_blocks = soup.find_all('div',{'class':'document-list__item document-list__item--col'})
  dimSource = {}

  for idx in range(len(xls_blocks)):
    title = xls_blocks[idx].find('div',{'class':'document-list__item-title'}).get_text().replace("\n", "").strip()
    content = xls_blocks[idx].find('a',{'class':'btn btn-icon btn-white btn-br btn-sm'}).get('href')
    download_link = connect_xls_url + content
    dimSource[title] = download_link

  return dimSource

In [18]:
dimSource = get_links_dict_2(soup)

In [19]:
def parse_df_2(link, sheet_name, skiprows, skipfooter):
  df = pd.read_excel(
    link, 
    sheet_name=sheet_name, 
    skiprows=skiprows,
    skipfooter=skipfooter
  )

  df.replace({'...': None}, inplace=True)

  return df

In [20]:
df_telecom_name = 'Среднемесячная номинальная начисленная заработная плата работников в целом по экономике по субъектам Российской Федерации за 2000-2021 гг.'
df_salary_2018_2021 = parse_df_2(
    dimSource[df_telecom_name], 
    sheet_name=1, 
    skiprows=1, 
    skipfooter=0 
).rename(columns={'Unnamed: 0': "region"})

df_salary_2000_2017 = parse_df_2(
    dimSource[df_telecom_name], 
    sheet_name=0, 
    skiprows=3, 
    skipfooter=2 
).rename(columns={'Unnamed: 0': "region"})


In [21]:
df_salary_2000_2021 = pd.concat([df_salary_2000_2017, df_salary_2018_2021[df_salary_2018_2021.columns[1:]]], axis=1)

In [22]:
years = list(df_salary_2000_2021.columns[1:])

In [23]:
final_df = df_salary_2000_2021[['region', years[0]]].rename(columns={years[0]: "value"})
final_df['year'] = years[0]
for x in years[1:]:
  current_df = df_salary_2000_2021[['region', x]].rename(columns={x: "value"})
  current_df['year'] = x
  final_df = pd.concat([final_df, current_df])

In [24]:
salary_to_load = final_df[~final_df['region'].str.contains('округ')]
salary_to_load = salary_to_load[~salary_to_load['region'].str.contains('Российская Федерация')]

In [25]:
salary_to_load = salary_to_load[~salary_to_load['region'].str.contains('в том числе')]

# Источник 3


*   Численность населения




In [26]:
df_population = parse_df_2(
    'https://rosstat.gov.ru/storage/mediabank/chisl_%D0%9C%D0%9E_Site_01-01-2022.xlsx', 
    sheet_name=1, 
    skiprows=6, 
    skipfooter=0 
)

In [27]:
Regs = parse_df_2(
    'https://rosstat.gov.ru/storage/mediabank/chisl_%D0%9C%D0%9E_Site_01-01-2022.xlsx', 
    sheet_name=0, 
    skiprows=0, 
    skipfooter=0 
)['Перечень субъектов РФ'].unique()

In [28]:
df_population = df_population.rename(columns={'Unnamed: 1':'region','(человек)':'population'})

In [29]:
df_population = df_population[['region','population']]
df_population['year'] = 2022

In [30]:
df_population = df_population[df_population['region'].isin(Regs)]

Сбор данных в один xlsx файл

In [32]:
writer = pd.ExcelWriter('./data/main.xlsx', engine='xlsxwriter')

#write each DataFrame to a specific sheet
df_population.to_excel(writer, sheet_name='Численность населения', index=False)
salary_to_load.to_excel(writer, sheet_name='Средняя зарплата', index=False)
telecom_to_load.to_excel(writer, sheet_name='Телеком', index=False)

#close the Pandas Excel writer and output the Excel file
writer.save()